In [2]:
import pandas as pd
import numpy as np
import os

In [17]:
fp = os.path.join(os.getcwd(),
                  'capitan_iceplant_pts.csv')
cap_ice_pts = pd.read_csv(fp)
# these are only 2020
sample_cap_ice = cap_ice_pts.sample(frac=0.3)  

# merge previously sampled train and test
train = pd.read_csv(os.path.join(os.getcwd(), 'LSWE_train_set.csv')) 
test = pd.read_csv(os.path.join(os.getcwd(), 'LSWE_test_set.csv'))
all_pts = pd.concat([train,test])

# choose only 2020 points
all_pts = all_pts[all_pts.year == 2020]

# iceplant: sample 1 pt from each polygn from each aoi
iceplant = all_pts[all_pts.iceplant==1]

ice_pts = iceplant.groupby(['aoi','polygon_id'], group_keys=False).apply(lambda df: df.sample(2))

# non-iceplant: sample 1 pt from each polygn from each aoi
nonice = all_pts[all_pts.iceplant==0]

nonice_pts = nonice.groupby(['aoi','polygon_id'], group_keys=False).apply(lambda df: df.sample(2))

# randomly sampled vegetation pts 
fp = os.path.join(os.getcwd(), 'vegetation_samples.csv')
extra_veg = pd.read_csv(fp)

all_veg = pd.concat([nonice_pts, extra_veg]).reset_index(drop=True)  #509 (1 pt/polygon) 
all_ice = pd.concat([ice_pts, sample_cap_ice]).reset_index(drop=True) #156 (1 pt/polygon)

In [5]:
#training: sample 70% of iceplant a vegetation pts from each aoi 
ice_train = all_ice.groupby(['aoi'], group_keys=False).apply(lambda df: df.sample(frac=0.7))
veg_train = all_veg.groupby(['aoi'], group_keys=False).apply(lambda df: df.sample(frac=0.7))

ice_test = all_ice.loc[list(set(range(len(all_ice))) - set(ice_train.index))]
veg_test = all_veg.loc[list(set(range(len(all_veg))) - set(veg_train.index))]

In [6]:
train = pd.concat([ice_train, veg_train])
test = pd.concat([ice_test, veg_test])

train.to_csv(os.path.join(os.getcwd(),'ptpoly2_train_2020.csv'), index=False)
test.to_csv(os.path.join(os.getcwd(),'ptpoly2_test_2020.csv'), index=False)